In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import random

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
# Define transformation
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.Resize((48, 48)),  # Resize images to 48x48
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize((0.5,), (0.5,))  # Normalize
])

# Load dataset
train_data = datasets.ImageFolder("/Users/shaneab/Projects/Machine Learning/Expression recognition/jonathanheix dataset/images/images/train", transform=transform)
loaded_train = DataLoader(train_data, batch_size=64, shuffle=True)

validation_data = datasets.ImageFolder("/Users/shaneab/Projects/Machine Learning/Expression recognition/jonathanheix dataset/images/images/validation", transform=transform)
loaded_validation = DataLoader(validation_data, batch_size=64, shuffle=False)


In [3]:
# Define CNN Model
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 512, kernel_size=(3,3), bias=True, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2))
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(512, 128, kernel_size=(3,3), padding=1, bias=True),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2))
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 64, kernel_size=(3,3), bias=True, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2))
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(64, 256, kernel_size=(3,3), bias=True, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2))
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256*3*3, 7)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.fc(x)
        return x


In [5]:
# Initialize model, loss, and optimizer
model = EmotionCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_part(model, optimizer, epochs=10):
    model = model.to(device)  # move the model parameters to CPU/GPU
    for epoch in range(epochs):
        print("Epoch:", epoch + 1)
        running_loss = 0.0
        for images, labels in loaded_train:
            images, labels = images.to(device), labels.to(device, dtype=torch.long)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loaded_train):.4f}")

print("Starting training...")
train_part(model, optimizer, epochs=10)
print("Training complete!")

Starting training...
Epoch: 1


KeyboardInterrupt: 

In [ ]:
# Visualization setup
%matplotlib inline

# Load a batch of training data
dataiter = iter(loaded_train)
images, labels = next(dataiter)

expression = {0: "angry", 1: "disgust", 2: "fear", 3: "happy", 4: "neutral", 5: "sad", 6: "surprise"}

# Display a random image and its label
random_idx = random.randint(0, 63)
print("Target label:", expression[int(labels[random_idx].cpu().numpy())])
plt.imshow(np.transpose(images[random_idx].cpu().numpy(), (1, 2, 0)), cmap='gray')
plt.show()
